In [2]:
import pandas as pd
import xmlrpc.client
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)


# TIEMPOS ESTANDAR

In [3]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name','workcenter_id']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['centro_produccion'] = test['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)


test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")

# Merge claficacion

test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard" , "display_name":"maquina_esperada"})

pareto_prod = pareto_prod[["nom_produccion","Clasificacion"]] #Columna realmente requeridas
te_clas = test.merge(pareto_prod, how="left",on="nom_produccion") 

# Descartar productos sin ventas ya que no tienen clasificacion
sin_ventas = te_clas[te_clas["Clasificacion"].isna()] 
te_clas = te_clas[~te_clas["nom_produccion"].isin(sin_ventas["nom_produccion"])]

te_clas = te_clas[["nom_produccion","producto","Clasificacion","centro_produccion","tiempo_estimado_estandard","maquina_esperada"]]


In [4]:
te_clas[te_clas["producto"].str.contains("DA4772A")]

,nom_produccion,producto,Clasificacion,centro_produccion,tiempo_estimado_estandard,maquina_esperada
125,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,CORTE,0.0,DESARROLLO Y ALTURA
563,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,CORTE,0.0,ENTUBADORA 4
2306,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,INTERNOS GUATA,0.0,GUATA
2726,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,INTERNOS GUATA,0.0,TERMOSELLADORA
3026,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,INYECCION,0.0,OMS
3631,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,BOLSA
4520,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,MARCADO/HOTMELT/ARANDELA/EMPAQUE
5278,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,CONTROL DE CALIDAD
5599,DA4772A,DA4772A F. AIRE INTERNO BOBCAT,AAA,EMBALAJE,0.0,EMBALAJE


# INFO OP

In [5]:
import xmlrpc.client
import pandas as pd
import re

# --- Configuración de conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- 1️⃣ Órdenes de producción terminadas ---
producciones_done = models.execute_kw(
    db, uid, password,
    "mrp.production", "search_read",
    [[("state", "=", "done")], ["id", "name"]]
)
df_producciones = pd.DataFrame(producciones_done)
ids_done = df_producciones["id"].tolist()

# --- 2️⃣ Work orders asociadas ---
workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line", "search_read",
    [
        [("production_id", "in", ids_done)],
        [
            "name", "production_id", "product_id", "product_internal_ref",
            "workcenter_id", "date_start", "date_finished", "hour",
            "tiempo_segundos_hombre", "qty", "user_start", "fuerza_laboral",
            "state", "delay", "tiempo_estimado_x_fuerza_lab",
            "tiempo_estimado_segundos","tiempo_real_segundos"
        ]
    ]
)

df_workorders = pd.DataFrame(workorders)

import re

def extraer_nom_produccion_desde_name(valor):
    """Extrae el código del producto (ej: DA4570A) desde la columna 'name'."""
    if not isinstance(valor, str):
        return None
    match = re.search(r'\bDA\d{3,5}[A-Z]?\b', valor)
    return match.group(0) if match else None

df_workorders["nom_produccion"] = df_workorders["name"].apply(extraer_nom_produccion_desde_name)


# --- 4️⃣ Limpieza y creación de columnas ---
df_workorders["product_id_name"] = df_workorders["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["product_id_int"] = df_workorders["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
)


df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["production_id_int"] = df_workorders["production_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) and len(x) > 0 else None
)
df_workorders["operario"] = df_workorders["user_start"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["centro_produccion"] = df_workorders["workcenter_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)
df_workorders["maquina_real"] = df_workorders["name"].str.split(" - ", n=1).str[0].str.strip()

df_workorders["nom_produccion"] = df_workorders["nom_produccion"].fillna("")


#TARDA 2 min

# TIEMPO OPERATIVO

In [6]:
import xmlrpc.client
import pandas as pd
from datetime import datetime

# ===============================
# 1️⃣ Conexión con Odoo 8
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# ===============================
# 2️⃣ Modelo a consultar
# ===============================
modelo = 'mrp.production.workcenter.line'  # cámbialo según necesites

# ===============================
# 3️⃣ Filtro: registros con fecha > 2023-01-01
# ===============================
fecha_inicio = '2023-01-01 00:00:00'
filtro = [('create_date', '>=', fecha_inicio)]

# ===============================
# 4️⃣ Consulta
# ===============================
records = models.execute_kw(
    db, uid, password,
    modelo, 'search_read',
    [filtro],
    {'limit': 1000000}
)

df_sample = pd.DataFrame(records)


In [34]:
df_sample[df_sample["OP"]=="MO13136"].head(1)

,origin,message_follower_ids,fecha_inicial_pause,create_date,es_contabilizado,sequence,production_state,tiempo_segundos_hombre,qty,fecha_final_pausa,tiempo_estimado_segundos,message_ids,user_finish,message_summary,create_uid,display_name,message_is_follower,__last_update,delay_pausa,date_start,message_last_post,state,id,delay,user_start,workcenter_id,uom,product,product_internal_ref,date_finished,date_planned_end,tiempo_optimo_segundos,tiempo_optimo_horas,write_date,tiempo_estimado_x_fuerza_lab,write_uid,cycle,message_unread,name,hour,date_planned,cantidad_hombres,product_id,production_id,website_message_ids,fuerza_laboral,OP
5430,OP22509024,[11156],False,2025-09-05 15:15:50,True,1,done,41776.44,1004.0,False,52012.5,"[12517325, 12517324]","[302, JAMER PUPO SIERRA]",,"[369, BRAYAN NAIR MUÑOZ SALAS]","ENTUBADORA 1 - DA2788 FILTRO AIRE DONSSON - Chevrolet NPR, NNR, NHR, NKR Reward",False,2025-09-18 20:15:07,0.0,2025-09-05 10:31:13,False,done,79244,1140239.0,"[302, JAMER PUPO SIERRA]","[54, CORTE]","[1, Und(es)]","[17162, [DAB02788025] DA2788 FILTRO AIRE DONSSON - Chevrolet NPR, NNR, NHR, NKR Reward (025 DA2788)]",DA2788B,2025-09-18 15:15:12,2025-09-06 10:27:00,45000.0,12.5,2025-09-18 20:15:07,17337.5,"[723, SIOMARA HERNANDEZ]",1000.0,False,"ENTUBADORA 1 - DA2788 FILTRO AIRE DONSSON - Chevrolet NPR, NNR, NHR, NKR Reward",14.45,2025-09-05 00:00:00,3.0,"[17162, [DAB02788025] DA2788 FILTRO AIRE DONSSON - Chevrolet NPR, NNR, NHR, NKR Reward (025 DA2788)]","[13407, MO13136]",[],"[110428, 110439, 110490]",MO13136


In [41]:
df_sample["OP"] = df_sample["production_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) and len(x) > 1 else None
)

#date_start inicio
#
ej = df_sample[df_sample["OP"]=="MO13136"].head(1)

#tiempo_segundos_hombre , tiempo_estimado _segundos son estimados  , last_update es la fecha de finalizacion de la orden
# delay_pause es el tiempo total de pausas en segundos toma el maximo si habian 2 en la op
# deay es el tiempo transcurrido 


ej =ej[["date_start","create_date","__last_update","date_finished","date_planned_end","write_date", 'fecha_final_pausa', 'fecha_inicial_pause',"user_finish"]]
ej.head()

,date_start,create_date,__last_update,date_finished,date_planned_end,write_date,fecha_final_pausa,fecha_inicial_pause,user_finish
5430,2025-09-05 10:31:13,2025-09-05 15:15:50,2025-09-18 20:15:07,2025-09-18 15:15:12,2025-09-06 10:27:00,2025-09-18 20:15:07,False,False,"[302, JAMER PUPO SIERRA]"


In [39]:
df_sample.columns.sort_values()

Index(['OP', '__last_update', 'cantidad_hombres', 'create_date', 'create_uid',
       'cycle', 'date_finished', 'date_planned', 'date_planned_end',
       'date_start', 'delay', 'delay_pausa', 'display_name',
       'es_contabilizado', 'fecha_final_pausa', 'fecha_inicial_pause',
       'fuerza_laboral', 'hour', 'id', 'message_follower_ids', 'message_ids',
       'message_is_follower', 'message_last_post', 'message_summary',
       'message_unread', 'name', 'origin', 'product', 'product_id',
       'product_internal_ref', 'production_id', 'production_state', 'qty',
       'sequence', 'state', 'tiempo_estimado_segundos',
       'tiempo_estimado_x_fuerza_lab', 'tiempo_optimo_horas',
       'tiempo_optimo_segundos', 'tiempo_segundos_hombre', 'uom',
       'user_finish', 'user_start', 'website_message_ids', 'workcenter_id',
       'write_date', 'write_uid'],
      dtype='object')

In [24]:
import xmlrpc.client
import pandas as pd
from datetime import datetime

# ===============================
# 1️⃣ Conexión con Odoo 8
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# ===============================
# 2️⃣ Modelo a consultar
# ===============================
modelo = 'mrp.report.verificacion.fuerza.labcon'  # cámbialo según necesites

# ===============================
# 3️⃣ Filtro: registros con fecha > 2023-01-01
# ===============================
fecha_inicio = '2023-01-01 00:00:00'
filtro = [('create_date', '>=', fecha_inicio)]

# ===============================
# 4️⃣ Consulta
# ===============================
fl = models.execute_kw(
    db, uid, password,
    modelo, 'search_read',
    [filtro],
    {'limit': 10000000}
)

df_fl = pd.DataFrame(fl)


In [30]:
df_fl.head()

,create_uid,create_date,operario,operacion,__last_update,fecha_final,tiempo_trabajo_unitario,rvfl_id,fecha_inicial,state,tiempo_trabajo,fuerza_id,tiempo_unitario_anterior,write_date,display_name,tiempo_trabajo_hrs,write_uid,id
0,"[1, Administrador Administrador ]",2025-02-20 14:47:25,"[459, ERNESTO VALENCIA ORJUELA]","[68289, REPUJADO - DA4133 FILTRO AIRE 2_ JOHN DEERE]",2025-02-20 14:47:25,False,0.000000,"[15654, mrp.report.verificacion.fuerza.lab,15654]",2025-02-20 08:30:27,startworking,0.0,"[94967, REPUJADO - DA4133 FILTRO AIRE 2_ JOHN DEERE]",206.400000,2025-02-20 14:47:25,ERNESTO VALENCIA ORJUELA,0.000000,"[1, Administrador Administrador ]",51335
1,"[1, Administrador Administrador ]",2025-02-20 14:47:25,"[292, ALVARO MELO PRADA]","[68288, TAPAS - DA4133 FILTRO AIRE 2_ JOHN DEERE]",2025-02-20 14:47:25,False,0.000000,"[15654, mrp.report.verificacion.fuerza.lab,15654]",2025-02-18 08:59:22,pause,0.0,"[94781, TAPAS - DA4133 FILTRO AIRE 2_ JOHN DEERE]",80.128571,2025-02-20 14:47:25,ALVARO MELO PRADA,0.000000,"[1, Administrador Administrador ]",51336
2,"[1, Administrador Administrador ]",2025-02-20 14:47:25,"[303, JOSE DANIEL PUPO SIERRA]","[68290, ENTUBADORA 4 - DA4133 FILTRO AIRE 2_ JOHN DEERE]",2025-02-20 14:47:25,2025-02-19 14:30:16,17.783333,"[15654, mrp.report.verificacion.fuerza.lab,15654]",2025-02-19 10:47:37,done,1067.0,"[94900, ENTUBADORA 4 - DA4133 FILTRO AIRE 2_ JOHN DEERE]",62.692857,2025-02-20 14:47:25,JOSE DANIEL PUPO SIERRA,0.296389,"[1, Administrador Administrador ]",51337
3,"[1, Administrador Administrador ]",2025-02-20 14:47:25,"[549, FRANCY CAROLINA PINZON CHACON]","[68260, ENTUBADORA 2 - DA8059 FILTRO AIRE 1o VOLVO,NEW HOLLAND]",2025-02-20 14:47:25,2025-02-19 07:31:40,82.100000,"[15655, mrp.report.verificacion.fuerza.lab,15655]",2025-02-18 14:57:41,done,1642.0,"[94844, ENTUBADORA 2 - DA8059 FILTRO AIRE 1o VOLVO,NEW HOLLAND]",105.575000,2025-02-20 14:47:25,FRANCY CAROLINA PINZON CHACON,0.456111,"[1, Administrador Administrador ]",51338
4,"[1, Administrador Administrador ]",2025-02-20 14:47:25,"[279, BLANCA DORA CUBIDES SANCHEZ]","[68259, ENTUBADORA 3 - DA8059 FILTRO AIRE 1o VOLVO,NEW HOLLAND]",2025-02-20 14:47:25,2025-02-19 07:00:55,85.450000,"[15655, mrp.report.verificacion.fuerza.lab,15655]",2025-02-18 15:19:03,done,1709.0,"[94851, ENTUBADORA 3 - DA8059 FILTRO AIRE 1o VOLVO,NEW HOLLAND]",236.500000,2025-02-20 14:47:25,BLANCA DORA CUBIDES SANCHEZ,0.474722,"[1, Administrador Administrador ]",51339


# AGRUPACION

In [8]:
import pandas as pd

# Copiamos el dataset original
df = df_aaa.copy()
df["date_finished"] = pd.to_datetime(df["date_finished"], errors="coerce")

# 🔹 Filtrar fechas desde 2023
df = df[df["date_finished"] >= pd.Timestamp("2023-01-01")].copy()

# 🔹 Determinar fila representativa (máquina esperada o mayor estándar)
cols_base = ["nom_produccion", "OP", "date_finished", "operario"]
df["match_maquina"] = df["maquina_real"] == df["maquina_esperada"]

def seleccionar_fila(grupo):
    coincidentes = grupo[grupo["match_maquina"]]
    if not coincidentes.empty:
        return coincidentes.iloc[0]
    grupo_pos = grupo[grupo["tiempo_estimado_estandard"] > 0]
    if not grupo_pos.empty:
        return grupo_pos.sort_values("tiempo_estimado_estandard", ascending=False).iloc[0]
    return grupo.iloc[0]

df_filtrado = (
    df.groupby(cols_base, group_keys=False)
    .apply(seleccionar_fila)
    .reset_index(drop=True)
)

# 🔹 Calcular tiempo estándar total por producto (solo 1 valor por nom_produccion)
tiempos_estandar = (
    df_filtrado.groupby("nom_produccion", as_index=False)
    .agg({"tiempo_estimado_estandard": "max"})  # usamos MAX para dejarlo fijo
    .rename(columns={"tiempo_estimado_estandard": "tiempo_estandar_fijo"})
)

# 🔹 Consolidar producción
cols_group = [
    "nom_produccion", "OP", "date_finished",
    "centro_produccion", "maquina_real",
    "Clasificacion", "producto", "maquina_esperada", "tasa_prod_real"
]

df_final = (
    df_filtrado.groupby(cols_group, as_index=False)
    .agg({
        "qty": "sum",
        "tiempo_segundos_hombre": "sum",
        "operario": lambda x: ", ".join(sorted(x.unique()))
    })
)

# 🔹 Asociar el tiempo estándar fijo a cada producto
df_final = df_final.merge(tiempos_estandar, on="nom_produccion", how="left")

# 🔹 Asegurarse que no se repita en PBI
df_final["tiempo_estimado_estandard"] = df_final["tiempo_estandar_fijo"]

# 🔹 (opcional) Calcular eficiencia
df_final["eficiencia_%"] = (
    df_final["tasa_prod_real"] / df_final["tiempo_estandar_fijo"] * 100
).round(2)

print(df_final[["nom_produccion", "OP", "tiempo_estimado_estandard"]].drop_duplicates())




NameError: name 'df_aaa' is not defined

In [ ]:
df_final.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/tiempo_estimado_estandard.xlsx")